In [1]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt 
import missingno as msno
import seaborn as sns 
import os

In [2]:
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})

In [3]:
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
[fm.FontProperties(fname=font).get_name() for font in font_list if 'D2C' in font]
plt.rc('font', family='D2Coding')
mpl.rcParams['axes.unicode_minus'] = False

In [4]:
DATASET_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/dataset/'
load_dir_name = 'america_big_cities_health_inventory'
file_name = 'BigCitiesHealth.csv'
RSLT_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/1.project1_structured/BCHI/processed/'

## FUNCTIONS

- preprocess

In [5]:
def count_key_opt(data:pd.DataFrame,key,opt):
    rslt = dict()
    labels = data[key].unique()
    form = data[opt].value_counts().sort_values(ascending=False)
    form.iloc[:] = 0

    for feat in labels:
        cond = data[key]==feat
        val = form.copy()
        temp = data.loc[cond,opt].value_counts()
        val.loc[temp.index] = temp
        rslt[feat] = val

    return pd.DataFrame(rslt).T

- plot

In [6]:
import re

def choose_split_point(word_len,space,ths):
    # 윗 줄에 space 만큼 공백이 있고, 한 줄의 길이가 ths로 제한 되어있을 때
    # 어떤 지점에서 단어를 끊어줄지 정하기
    # |-------ths-------|
    # |-space-|---------|-space-|------| : word
    #         |-------ths-------|
    print(word_len,space,ths)
    if word_len < ths + space :
        if abs(word_len/2 -ths) <= abs(word_len/2-space) :
            return word_len-ths
        else :
            return word_len - space if word_len < 2 * space else space
    else :
        return ths if word_len - (ths + space) < 0.3 * ths else space

def minimize_seq_idx_np(domain:np.array,func):
    vfunc = np.vectorize(func)
    temp = np.argsort(vfunc(domain))
    return temp[0]

def modify_strlen_ths(last,new,ths=16):
    front = len(last)
    space = ths - (1+front)
    if len(new) < space :
        rslt = [last + ' ' + new]
    else :
        if len(new) < ths:
            rslt = [last, new]
        else:
            cut = choose_split_point(len(new),space-1,ths-1)
            new_h, new_e = new[:cut]+'-', new[cut:]
            if cut < ths-1 :
                rslt = modify_strlen_ths(last+' '+new_h,new_e)
            else :
                rslt = [last] + modify_strlen_ths(new_h,new_e) 
    return rslt

def str_cutter(sentnc, ths = 16):
    words= sentnc.split(' ')
    rslt, pnt = [''], 0
    while pnt < len(words):
        last = '' if len(rslt)==0 else rslt[-1]
        next_ele = modify_strlen_ths(last,words[pnt],ths)
        rslt = rslt[:-1] + next_ele
        pnt += 1
    return '\n'.join(rslt)[1:]
#알고리즘 때문에 맨 앞에 빈칸 하나 들어가게 되는 이슈 있음

print(str_cutter('Racial Segregation Indices | Racial Segregation, White and Hispanic', 13))



Racial
Segregation
Indices |
Racial
Segregation,
White and
Hispanic


In [7]:
def choose_plot_grid(n:int,r_max=8,c_max=17):
    rs = np.sqrt(n)
    r_min = np.ceil(n/c_max)
    sppt = np.arange(r_min,r_max+1) #need error process
    col_nums = np.ceil(n/sppt)
    res = col_nums * sppt -n
    min_idx = np.where(res==np.min(res))
    #TODO : if len(min_idx) > 1 : minimize abs(sppt-col_nums), col_nums
    return sppt[min_idx[0]], col_nums[min_idx[0]]


def plot_feat_hue(data:pd.DataFrame,hue_label_dict=None):
    num_r, num_c = choose_plot_grid(len(data))
    fig, axes = plt.subplots(num_r,num_c,figsize=(21,17),sharex=True,sharey=True)
    for n,ax in enumerate(axes.flatten()):
        plt.setp(ax.get_xticklabels(),ha = 'left',rotation = 90)
        if n >= len(data) : continue
        sns.barplot(x=data.iloc[n].index, y = data.iloc[n].values,ax =ax)
        feat_name = data.index[n]
        if hue_label_dict: color = 'b' if hue_label_dict[feat_name] else 'k'
        else : color = 'k'
        ax.set_xlabel(str_cutter(feat_name,20),loc='left',fontsize = 8.3,color=color)

    pass

In [8]:
file_path = os.path.join(DATASET_DIR,load_dir_name,file_name)
df = pd.read_csv(file_path)

df.info()

/tmp/ipykernel_223497/3563803696.py:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189979 entries, 0 to 189978
Data columns (total 31 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   metric_item_label                189979 non-null  object 
 1   metric_cat_label                 189979 non-null  object 
 2   metric_subcat_label              189979 non-null  object 
 3   metric_item_label_subtitle       189979 non-null  object 
 4   metric_cat_item_yaxis_label      189979 non-null  object 
 5   metric_source_desc_label_fn      189979 non-null  object 
 6   metric_source_desc_label_url_fn  189979 non-null  object 
 7   geo_label_city                   189979 non-null  object 
 8   geo_label_state                  188667 non-null  object 
 9   geo_label_citystate              189979 non-null  object 
 10  geo_fips_code                    188667 non-null  float64
 11  value                            189979 non-null  float64
 12  da

In [9]:
df["metric_label"] = df["metric_subcat_label"]+' | '+df["metric_item_label"] 

In [10]:
cond1 = df['strata_race_label'] == 'Asian'

df_racesex=df[['strata_race_label','strata_sex_label','metric_label','geo_label_city','date_label','value']]

df_racesex.loc[cond1,'strata_race_label'] = 'Asian/PI'

cond1 = df['strata_race_label'] == 'American Indian/Alaska Native'
df_racesex.loc[cond1,'strata_race_label'] = 'Natives'


In [11]:
df_racesex['metric_label'].value_counts()

metric_label
Deaths | Premature Death                                  6825
Deaths | Injury Deaths                                    6405
Deaths | Deaths from All Causes                           6244
Cancer | All Cancer Deaths                                6233
Cardiovascular Disease | Cardiovascular Disease Deaths    6195
                                                          ... 
Housing | Homeless, Total                                  266
Housing | Vacant Housing and Homelessness                  266
Housing | Homeless, Children                               256
Respiratory Infection | COVID-19 Vaccination, Adults        70
Respiratory Infection | COVID-19 Vaccination, Seniors       70
Name: count, Length: 118, dtype: int64

## Before making pivot table

- agg 전에 race, sex, city, year, label 별로 unique 한지

In [12]:
df_racesex.columns

Index(['strata_race_label', 'strata_sex_label', 'metric_label',
       'geo_label_city', 'date_label', 'value'],
      dtype='object')

In [13]:
cols = list(df_racesex.columns)[:-1]
check_rslt = df_racesex.duplicated(subset=cols)
print(np.sum(check_rslt),'/',len(check_rslt))

0 / 189979


#### cf. 추가 EDA
##### race, sex, label, city 기준 하나씩 밖에 없는 데이터

In [14]:
check_rslt = df_racesex.duplicated(subset=cols[-1])
df_racesex[~check_rslt]

,strata_race_label,strata_sex_label,metric_label,geo_label_city,date_label,value
0,Black,Female,Cancer | All Cancer Deaths,Cleveland,2014,209.595434
1,Hispanic,Male,Cancer | All Cancer Deaths,Oakland,2016,162.848255
2,All,Female,Cancer | All Cancer Deaths,San Francisco,2021,96.706067
3,Hispanic,Female,Cancer | All Cancer Deaths,Long Beach,2018,104.393295
4,Black,Male,Cancer | All Cancer Deaths,Cleveland,2013,297.171301
5,White,Female,Cancer | All Cancer Deaths,Dallas,2010,151.115278
8,Black,Female,Cancer | All Cancer Deaths,San Antonio,2022,202.205060
9,Hispanic,Both,Cancer | All Cancer Deaths,Columbus,2011,91.778076
12,Asian/PI,Both,Cancer | All Cancer Deaths,Tucson,2015,107.470476
16,White,Female,Cancer | All Cancer Deaths,Long Beach,2019,146.532030


- label, city, year 기준 하나씩 밖에 없는 데이터

In [15]:
check_rslt = df_racesex.duplicated(subset=cols[2:])
df_thin_label = df_racesex[~check_rslt]
df_thin_label

,strata_race_label,strata_sex_label,metric_label,geo_label_city,date_label,value
0,Black,Female,Cancer | All Cancer Deaths,Cleveland,2014,209.595434
1,Hispanic,Male,Cancer | All Cancer Deaths,Oakland,2016,162.848255
2,All,Female,Cancer | All Cancer Deaths,San Francisco,2021,96.706067
3,Hispanic,Female,Cancer | All Cancer Deaths,Long Beach,2018,104.393295
4,Black,Male,Cancer | All Cancer Deaths,Cleveland,2013,297.171301
...,...,...,...,...,...,...
189974,All,Both,Racial Segregation Indices | Racial Segregatio...,Chicago,2010,60.194312
189975,All,Both,Racial Segregation Indices | Racial Segregatio...,Charlotte,2010,54.301191
189976,All,Both,Racial Segregation Indices | Racial Segregatio...,Boston,2010,55.285524
189977,All,Both,Racial Segregation Indices | Racial Segregatio...,Baltimore,2010,50.466207


In [16]:
cond0 = ~(df_thin_label[cols[0]] == 'All')
cond1 = ~(df_thin_label[cols[1]] == 'Both')

cond_ok = (~cond0) & (~cond1)
print('only for whole race-sex stats : ',len(df_thin_label[cond_ok]),'/',len(df_thin_label))

print("only for a race : ",len(df_thin_label[cond0]),'/',len(df_thin_label))
print("only for a sex : ",len(df_thin_label[cond1]),'/',len(df_thin_label))

cond = cond0 & cond1
print("only for a race-sex : ",len(df_thin_label[cond]),'/',len(df_thin_label))
df_thin_label.loc[cond,cols[2]].value_counts()

only for whole race-sex stats :  31308 / 46749
only for a race :  13271 / 46749
only for a sex :  4564 / 46749
only for a race-sex :  2394 / 46749


metric_label
Mental Health | Suicide                          449
Deaths | Premature Death                         418
Crime Incidents | Homicides                      318
Substance Use | Opioid Overdose Deaths           265
Cancer | All Cancer Deaths                       250
Deaths | Deaths from All Causes                  211
Cancer | Colorectal Cancer Deaths                189
Substance Use | Drug Overdose Deaths             126
Respiratory Infection | COVID-19 Deaths           45
Deaths | Motor Vehicle Deaths                     33
Deaths | Injury Deaths                            27
Cardiovascular Disease | Heart Disease Deaths     25
Diabetes and Obesity | Diabetes Deaths            24
Deaths | Gun Deaths (Firearms)                    14
Name: count, dtype: int64

- label, city 별로 하나 밖에 없는 데이터

In [17]:
check_rslt = df_racesex.duplicated(subset=cols[2:4])
df_thin_label = df_racesex[~check_rslt]
display(df_thin_label)
display(df_thin_label[cols[2]].value_counts())
display(df_thin_label[cols[3]].value_counts())

,strata_race_label,strata_sex_label,metric_label,geo_label_city,date_label,value
0,Black,Female,Cancer | All Cancer Deaths,Cleveland,2014,209.595434
1,Hispanic,Male,Cancer | All Cancer Deaths,Oakland,2016,162.848255
2,All,Female,Cancer | All Cancer Deaths,San Francisco,2021,96.706067
3,Hispanic,Female,Cancer | All Cancer Deaths,Long Beach,2018,104.393295
5,White,Female,Cancer | All Cancer Deaths,Dallas,2010,151.115278
...,...,...,...,...,...,...
189542,All,Both,Racial Segregation Indices | Racial Segregatio...,Chicago,2022,58.434179
189543,All,Both,Racial Segregation Indices | Racial Segregatio...,Charlotte,2022,55.088016
189544,All,Both,Racial Segregation Indices | Racial Segregatio...,Boston,2022,53.781505
189545,All,Both,Racial Segregation Indices | Racial Segregatio...,Baltimore,2022,46.920883


metric_label
Cancer | All Cancer Deaths                     36
Air Pollution | Poor Air Quality               36
Transportation | Drives Alone to Work          36
Active Transportation | Riding Bike to Work    36
Active Transportation | Walking to Work        36
                                               ..
School Violence | Weapons in School            18
Dietary Quality | Teen Breakfast               18
Mental Health | Teen Suicidal Ideation         18
Birth Control | Teen Birth Control             18
Physical Activity | Teen Physical Education    17
Name: count, Length: 118, dtype: int64

geo_label_city
Philadelphia     118
San Diego        118
Baltimore        118
Chicago          118
Detroit          118
Los Angeles      118
Charlotte        118
Boston           118
San Francisco    117
Washington       117
New York City    117
Fort Worth       117
Oakland          116
Cleveland        116
Seattle          116
Houston          115
Dallas           115
Portland         115
U.S. Total       110
Las Vegas        105
Columbus          99
Indianapolis      99
Phoenix           99
Milwaukee         99
Memphis           99
Louisville        99
Minneapolis       99
Oklahoma City     99
Denver            98
San Antonio       98
San Jose          98
Austin            98
Long Beach        98
El Paso           98
Tucson            97
Kansas City       96
Name: count, dtype: int64

In [18]:
cond0 = ~(df_thin_label[cols[0]] == 'All')
cond1 = ~(df_thin_label[cols[1]] == 'Both')

cond_ok = (~cond0) & (~cond1)
print('only for whole race-sex stats : ',len(df_thin_label[cond_ok]),'/',len(df_thin_label))

print("only for a race : ",len(df_thin_label[cond0]),'/',len(df_thin_label))
print("only for a sex : ",len(df_thin_label[cond1]),'/',len(df_thin_label))

cond = cond0 & cond1
print("only for a race-sex : ",len(df_thin_label[cond]),'/',len(df_thin_label))
df_thin_label.loc[cond,cols[2]].value_counts()

only for whole race-sex stats :  2540 / 3893
only for a race :  1196 / 3893
only for a sex :  365 / 3893
only for a race-sex :  208 / 3893


metric_label
Mental Health | Suicide                    35
Substance Use | Drug Overdose Deaths       35
Deaths | Premature Death                   32
Crime Incidents | Homicides                21
Cancer | All Cancer Deaths                 20
Respiratory Infection | COVID-19 Deaths    20
Deaths | Deaths from All Causes            16
Substance Use | Opioid Overdose Deaths     15
Cancer | Colorectal Cancer Deaths          14
Name: count, dtype: int64

In [19]:
df_thin_label.loc[cond,cols[3]].value_counts()

geo_label_city
Philadelphia     8
Los Angeles      8
Denver           8
Dallas           7
New York City    7
Phoenix          7
Austin           7
Oakland          7
Boston           7
Fort Worth       7
Washington       6
Milwaukee        6
Louisville       6
Houston          6
Cleveland        6
San Diego        6
Las Vegas        6
Indianapolis     6
Seattle          6
San Antonio      6
Long Beach       6
Memphis          6
Tucson           5
Charlotte        5
Minneapolis      5
San Jose         5
San Francisco    5
El Paso          5
Baltimore        5
Chicago          5
Detroit          5
Portland         5
Kansas City      5
Columbus         4
Oklahoma City    4
Name: count, dtype: int64

In [20]:
df_thin_label.loc[cond,cols[4]].value_counts()

date_label
2013    32
2014    26
2012    23
2017    21
2016    20
2021    19
2020    16
2010    13
2018    10
2022    10
2011     7
2019     6
2015     5
Name: count, dtype: int64

- label, year 별로 하나 밖에 없는 데이터

In [21]:
print(cols[2::2])
check_rslt = df_racesex.duplicated(subset=cols[2::2])
df_thin_label = df_racesex[~check_rslt]
display(df_thin_label)
display(df_thin_label[cols[2]].value_counts())
display(df_thin_label[cols[3]].value_counts())

['metric_label', 'date_label']


,strata_race_label,strata_sex_label,metric_label,geo_label_city,date_label,value
0,Black,Female,Cancer | All Cancer Deaths,Cleveland,2014,209.595434
1,Hispanic,Male,Cancer | All Cancer Deaths,Oakland,2016,162.848255
2,All,Female,Cancer | All Cancer Deaths,San Francisco,2021,96.706067
3,Hispanic,Female,Cancer | All Cancer Deaths,Long Beach,2018,104.393295
4,Black,Male,Cancer | All Cancer Deaths,Cleveland,2013,297.171301
...,...,...,...,...,...,...
189799,All,Both,Racial Segregation Indices | Racial Segregatio...,U.S. Total,2014,50.890911
189835,All,Both,Racial Segregation Indices | Racial Segregatio...,U.S. Total,2013,50.890911
189871,All,Both,Racial Segregation Indices | Racial Segregatio...,U.S. Total,2012,52.438849
189907,All,Both,Racial Segregation Indices | Racial Segregatio...,U.S. Total,2011,52.438849


metric_label
Cancer | All Cancer Deaths                               13
Mental Health | Suicide                                  13
Air Pollution | Poor Air Quality                         13
Transportation | Drives Alone to Work                    13
Active Transportation | Riding Bike to Work              13
                                                         ..
Housing | Vacant Housing and Homelessness                 8
Births | Preterm Births                                   7
Respiratory Infection | COVID-19 Deaths                   3
Respiratory Infection | COVID-19 Vaccination, Seniors     2
Respiratory Infection | COVID-19 Vaccination, Adults      2
Name: count, Length: 118, dtype: int64

geo_label_city
U.S. Total       293
Milwaukee        130
Las Vegas        112
Seattle           98
Houston           52
Phoenix           48
New York City     41
Long Beach        41
Dallas            38
Tucson            35
Washington        32
Louisville        31
Oakland           30
Chicago           28
Los Angeles       28
Boston            26
Detroit           26
Baltimore         26
Philadelphia      25
San Diego         23
Indianapolis      21
Fort Worth        21
San Francisco     20
Charlotte         17
Cleveland         17
San Jose          17
Memphis           16
Columbus          16
San Antonio       16
El Paso           14
Minneapolis       13
Austin            13
Denver            13
Kansas City       13
Portland          11
Oklahoma City      8
Name: count, dtype: int64

In [22]:
cond0 = ~(df_thin_label[cols[0]] == 'All')
cond1 = ~(df_thin_label[cols[1]] == 'Both')

cond_ok = (~cond0) & (~cond1)
print('only for whole race-sex stats : ',len(df_thin_label[cond_ok]),'/',len(df_thin_label))

print("only for a race : ",len(df_thin_label[cond0]),'/',len(df_thin_label))
print("only for a sex : ",len(df_thin_label[cond1]),'/',len(df_thin_label))

cond = cond0 & cond1
print("only for a race-sex : ",len(df_thin_label[cond]),'/',len(df_thin_label))
df_thin_label.loc[cond,cols[2]].value_counts()

only for whole race-sex stats :  918 / 1409
only for a race :  425 / 1409
only for a sex :  138 / 1409
only for a race-sex :  72 / 1409


metric_label
Deaths | Premature Death                   13
Mental Health | Suicide                    13
Cancer | All Cancer Deaths                  9
Deaths | Deaths from All Causes             9
Crime Incidents | Homicides                 8
Substance Use | Opioid Overdose Deaths      8
Cancer | Colorectal Cancer Deaths           5
Substance Use | Drug Overdose Deaths        5
Respiratory Infection | COVID-19 Deaths     2
Name: count, dtype: int64

In [23]:
df_thin_label.loc[cond,cols[3]].value_counts()

geo_label_city
Las Vegas        11
Louisville        5
Seattle           5
New York City     4
Washington        4
Cleveland         4
Denver            4
Detroit           3
Dallas            3
Milwaukee         3
Chicago           2
Long Beach        2
San Antonio       2
San Jose          2
Austin            2
Columbus          2
San Diego         2
Philadelphia      2
Tucson            1
Fort Worth        1
San Francisco     1
Indianapolis      1
Los Angeles       1
Oklahoma City     1
Charlotte         1
Oakland           1
Minneapolis       1
Boston            1
Name: count, dtype: int64

In [24]:
df_thin_label.loc[cond,cols[4]].value_counts()

date_label
2010    7
2022    7
2020    7
2012    7
2016    6
2013    6
2017    6
2014    5
2018    5
2019    5
2011    5
2021    4
2015    2
Name: count, dtype: int64

## Makin pivot tables

1. only for whole race-sex stats

In [26]:
cond_total_race = (df_racesex[cols[0]] == 'All')
cond_total_sex = (df_racesex[cols[1]] == 'Both')
cond_total_area = (df_racesex[cols[3]] == 'U.S. Total')

cond_total_racesex = (cond_total_race) & (cond_total_sex)

df_whole_total = df_racesex[cond_total_racesex]
df_city_total = df_racesex[cond_total_racesex & (~cond_total_area)]
df_US_total = df_racesex[cond_total_racesex & (cond_total_area)]

print(len(df_whole_total))
print(len(df_city_total))
print(len(df_US_total))


46749
45437
1312


In [27]:
index_cols = ['geo_label_city','date_label']

pvtb_whole_total = pd.pivot_table(df_whole_total, index = index_cols, columns=['metric_label'],values=['value'])
pvtb_city_total = pd.pivot_table(df_city_total, index = index_cols, columns=['metric_label'],values=['value'])
pvtb_US_total = pd.pivot_table(df_US_total, index = index_cols, columns=['metric_label'],values=['value'])

display(pvtb_whole_total)
display(pvtb_city_total)
display(pvtb_US_total)

value  \
metric_label              Active Transportation | Riding Bike to Work   
geo_label_city date_label                                               
Austin         2010                                          1.458997   
               2011                                          1.458997   
               2012                                          1.458997   
               2013                                          1.326730   
               2014                                          1.326730   
...                                                               ...   
Washington     2018                                          3.316120   
               2019                                          3.316120   
               2020                                          3.316120   
               2021                                          3.316120   
               2022                                          3.316120   

                                                                   \
metric_label              Active Transportation | Walking to Work   
geo_label_city date_label                                           
Austin         2010                                      2.506039   
               2011                                      2.506039   
               2012                                      2.506039   
               2013                                      2.281223   
               2014                                      2.281223   
...                                                           ...   
Washington     2018                                     10.266102   
               2019                                     10.266102   
               2020                                     10.266102   
               2021                                     10.266102   
               2022                                     10.266102   

                                                                 \
metric_label              Air Pollution | Hazardous Air Quality   
geo_label_city date_label                                         
Austin         2010                                    2.191781   
               2011                                    3.835616   
               2012                                    2.185792   
               2013                                    1.095890   
               2014                                    0.000000   
...                                                         ...   
Washington     2018                                    1.643836   
               2019                                    1.369863   
               2020                                    0.273224   
               2021                                    2.191781   
               2022                                    1.877934   

                                                            \
metric_label              Air Pollution | Poor Air Quality   
geo_label_city date_label                                    
Austin         2010                              25.479452   
               2011                              38.356164   
               2012                              31.967213   
               2013                              31.780822   
               2014                              31.780822   
...                                                    ...   
Washington     2018                              35.616438   
               2019                              33.698630   
               2020                              14.207650   
               2021                              30.958904   
               2022                              28.169014   

                                                              \
metric_label              Birth Control | Teen Birth Control   
geo_label_city date_label                                      
Austin         2010                                      NaN   
          

value  \
metric_label              Active Transportation | Riding Bike to Work   
geo_label_city date_label                                               
Austin         2010                                          1.458997   
               2011                                          1.458997   
               2012                                          1.458997   
               2013                                          1.326730   
               2014                                          1.326730   
...                                                               ...   
Washington     2018                                          3.316120   
               2019                                          3.316120   
               2020                                          3.316120   
               2021                                          3.316120   
               2022                                          3.316120   

                                                                   \
metric_label              Active Transportation | Walking to Work   
geo_label_city date_label                                           
Austin         2010                                      2.506039   
               2011                                      2.506039   
               2012                                      2.506039   
               2013                                      2.281223   
               2014                                      2.281223   
...                                                           ...   
Washington     2018                                     10.266102   
               2019                                     10.266102   
               2020                                     10.266102   
               2021                                     10.266102   
               2022                                     10.266102   

                                                                 \
metric_label              Air Pollution | Hazardous Air Quality   
geo_label_city date_label                                         
Austin         2010                                    2.191781   
               2011                                    3.835616   
               2012                                    2.185792   
               2013                                    1.095890   
               2014                                    0.000000   
...                                                         ...   
Washington     2018                                    1.643836   
               2019                                    1.369863   
               2020                                    0.273224   
               2021                                    2.191781   
               2022                                    1.877934   

                                                            \
metric_label              Air Pollution | Poor Air Quality   
geo_label_city date_label                                    
Austin         2010                              25.479452   
               2011                              38.356164   
               2012                              31.967213   
               2013                              31.780822   
               2014                              31.780822   
...                                                    ...   
Washington     2018                              35.616438   
               2019                              33.698630   
               2020                              14.207650   
               2021                              30.958904   
               2022                              28.169014   

                                                              \
metric_label              Birth Control | Teen Birth Control   
geo_label_city date_label                                      
Austin         2010                                      NaN   
          

value  \
metric_label              Active Transportation | Riding Bike to Work   
geo_label_city date_label                                               
U.S. Total     2010                                          0.548428   
               2011                                          0.548428   
               2012                                          0.548428   
               2013                                          0.587269   
               2014                                          0.587269   
               2015                                          0.587269   
               2016                                          0.587269   
               2017                                          0.587269   
               2018                                          0.460903   
               2019                                          0.460903   
               2020                                          0.460903   
               2021                                          0.460903   
               2022                                          0.460903   

                                                                   \
metric_label              Active Transportation | Walking to Work   
geo_label_city date_label                                           
U.S. Total     2010                                      2.667280   
               2011                                      2.667280   
               2012                                      2.667280   
               2013                                      2.728075   
               2014                                      2.728075   
               2015                                      2.728075   
               2016                                      2.728075   
               2017                                      2.728075   
               2018                                      2.429932   
               2019                                      2.429932   
               2020                                      2.429932   
               2021                                      2.429932   
               2022                                      2.429932   

                                                                 \
metric_label              Air Pollution | Hazardous Air Quality   
geo_label_city date_label                                         
U.S. Total     2010                                    5.068512   
               2011                                    6.144475   
               2012                                    6.975557   
               2013                                    3.445394   
               2014                                    3.219178   
               2015                                    3.630477   
               2016                                    2.998975   
               2017                                    3.528589   
               2018                                    4.130990   
               2019                                    2.171535   
               2020                                    4.562917   
               2021                                    4.465866   
               2022                                    2.966538   

                                                            \
metric_label              Air Pollution | Poor Air Quality   
geo_label_city date_label                                    
U.S. Total     2010                              46.954181   
               2011                              52.178816   
               2012                              48.861564   
               2013                              45.263482   
               2014                              42.842466   
               2015                              41.553455   
               2016                              36.583452   
               2017                              38.548489   
               2018                

In [28]:

file_names=[
    'pvtb_whole_total_ver0.csv',
    'pvtb_city_total_ver0.csv',
    'pvtb_US_total_ver0.csv'
]

pvtb_dir = 'pvtb'

save_path=[
    os.path.join(RSLT_DIR,pvtb_dir,name)
    for name in file_names
]

pvtb_whole_total.to_csv(save_path[0],index=True)
pvtb_city_total.to_csv(save_path[1],index=True)
pvtb_US_total.to_csv(save_path[2],index=True)

2. for an entire data

In [29]:
cond_total_area = (df_racesex[cols[3]] == 'U.S. Total')

df_whole_entire = df_racesex
df_city_entire = df_racesex[~cond_total_area]
df_US_entire = df_racesex[cond_total_area]

print(len(df_whole_entire))
print(len(df_city_entire))
print(len(df_US_entire))


189979
188667
1312


In [30]:
index_cols = ['geo_label_city','date_label','strata_race_label','strata_sex_label']

pvtb_whole_entire = pd.pivot_table(df_whole_entire, index = index_cols, columns=['metric_label'],values=['value'])
pvtb_city_entire = pd.pivot_table(df_city_entire, index = index_cols, columns=['metric_label'],values=['value'])
pvtb_US_entire = pd.pivot_table(df_US_entire, index = index_cols, columns=['metric_label'],values=['value'])

display(pvtb_whole_entire)
display(pvtb_city_entire)
display(pvtb_US_entire)

value  \
metric_label                                                 Active Transportation | Riding Bike to Work   
geo_label_city date_label strata_race_label strata_sex_label                                               
Austin         2010       All               Both                                                1.458997   
                                            Female                                                   NaN   
                                            Male                                                     NaN   
                          Asian/PI          Both                                                     NaN   
                                            Female                                                   NaN   
...                                                                                                  ...   
Washington     2022       Hispanic          Male                                                     NaN   
                          Natives           Both                                                     NaN   
                          White             Both                                                     NaN   
                                            Female                                                   NaN   
                                            Male                                                     NaN   

                                                                                                      \
metric_label                                                 Active Transportation | Walking to Work   
geo_label_city date_label strata_race_label strata_sex_label                                           
Austin         2010       All               Both                                            2.506039   
                                            Female                                               NaN   
                                            Male                                                 NaN   
                          Asian/PI          Both                                                 NaN   
                                            Female                                               NaN   
...                                                                                              ...   
Washington     2022       Hispanic          Male                                                 NaN   
                          Natives           Both                                            8.766234   
                          White             Both                                           14.201808   
                                            Female                                               NaN   
                                            Male                                                 NaN   

                                                                                                    \
metric_label                                                 Air Pollution | Hazardous Air Quality   
geo_label_city date_label strata_race_label strata_sex_label                                         
Austin         2010       All               Both                                          2.191781   
                                            Female                                             NaN   
                                            Male                                               NaN   
                          Asian/PI          Both                                               NaN   
                                            Female                                             NaN   
...                                                                                            ...   
Washington     2022       Hispanic          Male                                               NaN   
                          Natives           Both                                               NaN   
       

value  \
metric_label                                                 Active Transportation | Riding Bike to Work   
geo_label_city date_label strata_race_label strata_sex_label                                               
Austin         2010       All               Both                                                1.458997   
                                            Female                                                   NaN   
                                            Male                                                     NaN   
                          Asian/PI          Both                                                     NaN   
                                            Female                                                   NaN   
...                                                                                                  ...   
Washington     2022       Hispanic          Male                                                     NaN   
                          Natives           Both                                                     NaN   
                          White             Both                                                     NaN   
                                            Female                                                   NaN   
                                            Male                                                     NaN   

                                                                                                      \
metric_label                                                 Active Transportation | Walking to Work   
geo_label_city date_label strata_race_label strata_sex_label                                           
Austin         2010       All               Both                                            2.506039   
                                            Female                                               NaN   
                                            Male                                                 NaN   
                          Asian/PI          Both                                                 NaN   
                                            Female                                               NaN   
...                                                                                              ...   
Washington     2022       Hispanic          Male                                                 NaN   
                          Natives           Both                                            8.766234   
                          White             Both                                           14.201808   
                                            Female                                               NaN   
                                            Male                                                 NaN   

                                                                                                    \
metric_label                                                 Air Pollution | Hazardous Air Quality   
geo_label_city date_label strata_race_label strata_sex_label                                         
Austin         2010       All               Both                                          2.191781   
                                            Female                                             NaN   
                                            Male                                               NaN   
                          Asian/PI          Both                                               NaN   
                                            Female                                             NaN   
...                                                                                            ...   
Washington     2022       Hispanic          Male                                               NaN   
                          Natives           Both                                               NaN   
       

value  \
metric_label                                                 Active Transportation | Riding Bike to Work   
geo_label_city date_label strata_race_label strata_sex_label                                               
U.S. Total     2010       All               Both                                                0.548428   
               2011       All               Both                                                0.548428   
               2012       All               Both                                                0.548428   
               2013       All               Both                                                0.587269   
               2014       All               Both                                                0.587269   
               2015       All               Both                                                0.587269   
               2016       All               Both                                                0.587269   
               2017       All               Both                                                0.587269   
               2018       All               Both                                                0.460903   
               2019       All               Both                                                0.460903   
               2020       All               Both                                                0.460903   
               2021       All               Both                                                0.460903   
               2022       All               Both                                                0.460903   

                                                                                                      \
metric_label                                                 Active Transportation | Walking to Work   
geo_label_city date_label strata_race_label strata_sex_label                                           
U.S. Total     2010       All               Both                                            2.667280   
               2011       All               Both                                            2.667280   
               2012       All               Both                                            2.667280   
               2013       All               Both                                            2.728075   
               2014       All               Both                                            2.728075   
               2015       All               Both                                            2.728075   
               2016       All               Both                                            2.728075   
               2017       All               Both                                            2.728075   
               2018       All               Both                                            2.429932   
               2019       All               Both                                            2.429932   
               2020       All               Both                                            2.429932   
               2021       All               Both                                            2.429932   
               2022       All               Both                                            2.429932   

                                                                                                    \
metric_label                                                 Air Pollution | Hazardous Air Quality   
geo_label_city date_label strata_race_label strata_sex_label                                         
U.S. Total     2010       All               Both                                          5.068512   
               2011       All               Both                                          6.144475   
               2012       All               Both                                          6.975557   
               2013       All               Both                                          3.4

In [31]:

file_names=[
    'pvtb_whole_entire_ver0.csv',
    'pvtb_city_entire_ver0.csv',
    'pvtb_US_entire_ver0.csv'
]

pvtb_dir = 'pvtb'

save_path=[
    os.path.join(RSLT_DIR,pvtb_dir,name)
    for name in file_names
]

pvtb_whole_entire.to_csv(save_path[0],index=True)
pvtb_city_entire.to_csv(save_path[1],index=True)
pvtb_US_entire.to_csv(save_path[2],index=True)

3. only for each stratification

In [33]:
cond_total_race = (df_racesex[cols[0]] == 'All')
cond_total_sex = (df_racesex[cols[1]] == 'Both')
cond_total_area = (df_racesex[cols[3]] == 'U.S. Total')

cond_strat_any = (~cond_total_race) | (~cond_total_sex)
cond_strat_all = (~cond_total_race) & (~cond_total_sex)

df_strat_any = df_racesex[cond_strat_any]
df_strat_all = df_racesex[cond_strat_all]

print(len(df_strat_any))
print(len(df_strat_all))


143230
46662


In [35]:
index_cols = ['geo_label_city','date_label','strata_race_label','strata_sex_label']

pvtb_strat_any = pd.pivot_table(df_strat_any, index = index_cols, columns=['metric_label'],values=['value'])
pvtb_strat_all = pd.pivot_table(df_strat_all, index = index_cols, columns=['metric_label'],values=['value'])

display(pvtb_strat_any)
display(pvtb_strat_all)

value  \
metric_label                                                 Active Transportation | Walking to Work   
geo_label_city date_label strata_race_label strata_sex_label                                           
Austin         2010       All               Female                                               NaN   
                                            Male                                                 NaN   
                          Asian/PI          Both                                                 NaN   
                                            Female                                               NaN   
                                            Male                                                 NaN   
...                                                                                              ...   
Washington     2022       Hispanic          Male                                                 NaN   
                          Natives           Both                                            8.766234   
                          White             Both                                           14.201808   
                                            Female                                               NaN   
                                            Male                                                 NaN   

                                                                                                 \
metric_label                                                 Birth Control | Teen Birth Control   
geo_label_city date_label strata_race_label strata_sex_label                                      
Austin         2010       All               Female                                          NaN   
                                            Male                                            NaN   
                          Asian/PI          Both                                            NaN   
                                            Female                                          NaN   
                                            Male                                            NaN   
...                                                                                         ...   
Washington     2022       Hispanic          Male                                            NaN   
                          Natives           Both                                            NaN   
                          White             Both                                            NaN   
                                            Female                                          NaN   
                                            Male                                            NaN   

                                                                                       \
metric_label                                                 Births | Low Birthweight   
geo_label_city date_label strata_race_label strata_sex_label                            
Austin         2010       All               Female                                NaN   
                                            Male                                  NaN   
                          Asian/PI          Both                             8.785406   
                                            Female                                NaN   
                                            Male                                  NaN   
...                                                                               ...   
Washington     2022       Hispanic          Male                                  NaN   
                          Natives           Both                                  NaN   
                          White             Both                             5.574537   
                                            Female                                NaN   
                                            Male                                  NaN   

    

value  \
metric_label                                                 Cancer | All Cancer Deaths   
geo_label_city date_label strata_race_label strata_sex_label                              
Austin         2010       Asian/PI          Female                            79.076751   
                                            Male                             158.041310   
                          Black             Female                           193.834961   
                                            Male                             305.996589   
                          Hispanic          Female                           103.754458   
...                                                                                 ...   
Washington     2022       Black             Male                             243.855425   
                          Hispanic          Female                            70.502925   
                                            Male                             106.913662   
                          White             Female                                  NaN   
                                            Male                                    NaN   

                                                                                                \
metric_label                                                 Cancer | Colorectal Cancer Deaths   
geo_label_city date_label strata_race_label strata_sex_label                                     
Austin         2010       Asian/PI          Female                                         NaN   
                                            Male                                           NaN   
                          Black             Female                                   19.475668   
                                            Male                                     21.027801   
                          Hispanic          Female                                   12.224394   
...                                                                                        ...   
Washington     2022       Black             Male                                     24.974548   
                          Hispanic          Female                                         NaN   
                                            Male                                           NaN   
                          White             Female                                    9.634692   
                                            Male                                           NaN   

                                                                                          \
metric_label                                                 Cancer | Lung Cancer Deaths   
geo_label_city date_label strata_race_label strata_sex_label                               
Austin         2010       Asian/PI          Female                                   NaN   
                                            Male                                     NaN   
                          Black             Female                             42.643078   
                                            Male                               66.930268   
                          Hispanic          Female                             17.241446   
...                                                                                  ...   
Washington     2022       Black             Male                               46.656841   
                          Hispanic          Female                                   NaN   
                                            Male                                     NaN   
                          White             Female                                   NaN   
                                            Male                                     NaN   

                                                                                                                     \
metric_label              

In [36]:
file_names=[
    'pvtb_strat_any_ver0.csv',
    'pvtb_strat_all_ver0.csv',
]

pvtb_dir = 'pvtb'

save_path=[
    os.path.join(RSLT_DIR,pvtb_dir,name)
    for name in file_names
]

pvtb_strat_any.to_csv(save_path[0],index=True)
pvtb_strat_all.to_csv(save_path[1],index=True)

### TODO

1. 결측치 확인
    1. for entire data 
    2. only for whole race-sex
    3. only for each stratification
2. 라벨별로 집계 주기 확인
    1. for entire data 
    2. only for whole race-sex
    3. only for each stratification